# Day 19: An Elephant Named Joseph

## Part One

The Elves contact you over a highly secure emergency channel. Back at the North Pole, the Elves are busy misunderstanding White Elephant parties.

Each Elf brings a present. They all sit in a circle, numbered starting with position 1. Then, starting with the first Elf, they take turns stealing all the presents from the Elf to their left. An Elf with no presents is removed from the circle and does not take turns.

For example, with five Elves (numbered 1 to 5):
```
  1
5   2
 4 3
```

* Elf 1 takes Elf 2's present.

* Elf 2 has no presents and is skipped.

* Elf 3 takes Elf 4's present.

* Elf 4 has no presents and is also skipped.

* Elf 5 takes Elf 1's two presents.

* Neither Elf 1 nor Elf 2 have any presents, so both are skipped.

* Elf 3 takes Elf 5's three presents.

So, with five Elves, the Elf that sits starting in position 3 gets all the presents.

With the number of Elves given in your puzzle input, which Elf gets all the presents?

---

In [1]:
# Initialise
inputs = 3_017_957

def get_elf_position(n):
    """
    This implements the analytical solution to the Josephus problem given n elves labelled from 1 to n.
    """
    L = n - 2**(n.bit_length() - 1)
    return 2*L + 1

# Solution
print(f"Elf {get_elf_position(inputs)} gets all the presents.")

Elf 1841611 gets all the presents.


---

## Part Two

Realizing the folly of their present-exchange rules, the Elves agree to instead steal presents from the Elf directly across the circle. If two Elves are across the circle, the one on the left (from the perspective of the stealer) is stolen from. The other rules remain unchanged: Elves with no presents are removed from the circle entirely, and the other elves move in slightly to keep the circle evenly spaced.

For example, with five Elves (again numbered `1` to `5`):

The Elves sit in a circle; Elf `1` goes first:
```
  1
5   2
 4 3
```

Elves `3` and `4` are across the circle; Elf `3`'s present is stolen, being the one to the left. Elf `3` leaves the circle, and the rest of the Elves move in:
```
  1           1
5   2  -->  5   2
 4 -          4
```

Elf `2` steals from the Elf directly across the circle, Elf `5`:
```
  1         1 
-   2  -->     2
  4         4 
```

Next is Elf `4` who, choosing between Elves `1` and `2`, steals from Elf `1`:
```
 -          2  
    2  -->
 4          4
```

Finally, Elf `2` steals from Elf `4`:
```
 2
    -->  2  
 -
```
So, with five Elves, the Elf that sits starting in position `2` gets all the presents.

With the number of Elves given in your puzzle input, which Elf now gets all the presents?

---

In [2]:
class IncompleteError(Exception):
    pass

class DoubleLinkedCircle:
    """
    This custom class is a double linked list where the initial node is 
    linked to the final node in order to create a circle. The successor 
    and previous methods will return the adjacent nodes which may change
    when items are deleted from the list.
    
    The input must be iterable and support list-like indexing. (E.g. lists,
    ranges, etc will be supported. Sets are not supported.)
    """
    def __init__(self, l):
        # Initialise dicts
        self.fwd_link = {}
        self.bwd_link = {}
        
        n = len(l)
        
        # Iterate over list-like object
        for i in range(n):
            # Set double links
            self.fwd_link[l[i]] = l[(i+1) % n]
            self.bwd_link[l[i]] = l[(i-1) % n]
    
    def __getitem__(self, item):
        """
        getitem will return a triple whose first element is the previous
        item in the linked list, then the item requested and then the next 
        item in the linked list.
        """
        return self.bwd_link[item], item, self.fwd_link[item]
    
    def __delitem__(self, item):
        """
        del will remove the given item (if possible) and then sew the
        list back together
        """
        if item in self.fwd_link:
            prev = self.bwd_link[item]
            succ = self.fwd_link[item]
            del self.bwd_link[item], self.fwd_link[item]
            self.fwd_link[prev] = succ
            self.bwd_link[succ] = prev
        else:
            raise KeyError(item)
    
    def __len__(self):
        return len(self.fwd_link)
    
    def __str__(self):
        """
        Displays 3 random items that are connected
        """
        start = self.previous(list(self.fwd_link.keys())[0])
        items = [start]
        for _ in range(min(2, len(self)-1)):
            items.append(self.successor(items[-1]))
            
        return f"DoubleLinkedCircle(..., {', '.join([str(item) for item in items])}, ...)"
    
    def __repr__(self):
        return str(self)
    
    def previous(self, i):
        """
        Returns the previous item in the linked circle
        """
        return self.bwd_link[i]
    
    def successor(self, i):
        """
        Returns the next (or successive) item in the linked circle
        """
        return self.fwd_link[i]
    
    def last(self):
        """
        If there is only one element in the linked circle it returns that,
        otherwise raises error. This is helpful for the AoC problem.
        """
        if len(self) == 1:
            return list(self.fwd_link.keys())[0]
        else:
            raise IncompleteError("This circle has more than one elf in it.")

In [3]:
# Initialise circle
elves = DoubleLinkedCircle(range(1,inputs+1))

# Initialise positions
position = 1
opposite = inputs//2 + 1
parity = inputs % 2

# We will only need to iterate a fixed number of times
for _ in range(inputs - 1):
    
    # Find the person that will be excluded from the circle
    kill = opposite
    # Progress the target (once or twice depending on parity)
    opposite = elves.successor(opposite)
    if parity:
        opposite = elves.successor(opposite)
    # Increment parity
    parity = (parity + 1) % 2
    
    # Remove elf from circle
    del elves[kill]
    
    # Progress to next elf in circle
    position = elves.successor(position)

# Solution
print(f"Elf {elves.last()} gets all the presents")

Elf 1423634 gets all the presents


---